# Detección de anomalías en la calidad del aire en dispositivos IoT en el Edge computing: un enfoque basado en datos de CO2

En este proyecto, utilizaremos TensorFlow para desarrollar un modelo de detección de anomalías para la calidad del aire. Nuestro enfoque consiste en la combinación de un Autoencoder y una Red Neuronal de Larga Memoria a Corto Plazo (LSTM) para identificar anomalías en los datos de CO2 recogidos por dispositivos IoT en el Edge computing.

Los datos que utilizaremos para entrenar y evaluar nuestro modelo provienen del CEIP Albea Valld'Alba, y están disponibles para su descarga en [Zenodo](https://doi.org/10.5281/zenodo.5036228). Este dataset contiene mediciones de la calidad del aire, incluyendo los niveles de CO2, y será la base para nuestro análisis y modelado.

A lo largo de este notebook, procesaremos y exploraremos estos datos, entrenaremos nuestro modelo de detección de anomalías, y evaluaremos su rendimiento. El objetivo final es desarrollar un modelo robusto y efectivo que pueda identificar anomalías en la calidad del aire en tiempo real, permitiendo una respuesta rápida a cualquier cambio en las condiciones del aire.

Guillem Campo Fons


In [2]:
#DATA ANALYSIS AND OPERATIONS
import numpy as np  
import pandas as pd

#TENSORFLOW
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, Dropout, RepeatVector

#SKLEARN
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, roc_auc_score, recall_score, confusion_matrix

#VISUALIZATIONS
from matplotlib import pyplot as plt
import seaborn as sns

import os

from numpy.random import seed
from scipy.stats import shapiro

TRATAMIENTO = 'ELIMINACION' #ELIMINACION o LIMITACION
NORMAL = 'ROBUST' #STANDARD o MINMAX o ROBUST
LOSS_FUNCTION = 'mae' #mse o mae
LOSS1 = 'MSE' #MAE o MSE
LOSS2 = 'MSE' #MAE o MSE

In [ ]:
df = pd.read_csv('CEIP_Albea_ValldAlba.csv', usecols=['date_time', 'co2', 'sensor_id'])
df['date_time'] = pd.to_datetime(df['date_time']).dt.strftime('%m-%d-%Y %H:%M')
df_sorted = df.sort_values(by='date_time')
sensor_ids = [' CO2_02']#, ' CO2_01', ' CO2_03', ' CO2_04', ' CO2_05', ' CO2_06']
df_filtered = []
for sensor_id in sensor_ids:
    df_filtered.append(df_sorted[df_sorted['sensor_id'] == sensor_id])
for i in range(len(df_filtered)):
    df = df_filtered[i].copy()
    df['date_time'] = pd.to_datetime(df['date_time'])
    df = df.drop_duplicates(subset='date_time', keep='first')
    df = df.drop(['sensor_id'], axis=1)
    df = df.set_index('date_time')
    df = df.resample('5T').asfreq()
    df = df.reset_index()
    df_filtered[i] = df
df_concatenated = pd.concat(df_filtered)
df_concatenated = (df_concatenated
                   .interpolate()
                   .drop('date_time', axis=1)
                   .reset_index(drop=True))
df_concatenated_nocap = df_concatenated.copy()

In [ ]:
sns.set(style="whitegrid")
fig, ax = plt.subplots(figsize=(18, 5))
sns.lineplot(data=df_concatenated, x=df_concatenated.index, y='co2', ax=ax, linewidth=2)
ax.set_xlabel('Order in Time', fontsize=12)
ax.set_ylabel('CO2 (PPM)', fontsize=12)
ax.set_title('CO2 Recordings Over Time by Sensor', fontsize=14)
ax.set_facecolor('#F5F5F5')
ax.grid(visible=True, linestyle='solid', alpha=0.5)
ax.margins(x=0.02, y=0.1)
plt.tight_layout()
plt.show()

In [ ]:
seed(1993)
stat, p = shapiro(df_concatenated)
print('Estadisticos=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
   print('La muestra parece Gaussiana o Normal (no se rechaza la hipótesis nula H0)')
else:
   print('La muestra no parece Gaussiana o Normal(se rechaza la hipótesis nula H0)')

In [ ]:
upper_limit = 968

if(TRATAMIENTO  == 'ELIMINACION'):
 df_concatenated = df_concatenated[df_concatenated['co2'] <= upper_limit]
if(TRATAMIENTO  == 'LIMITACION'):
 df_concatenated['co2'] = np.where(df_concatenated['co2'] > upper_limit, upper_limit, df_concatenated['co2'])


In [ ]:
sns.set(style="whitegrid")
fig, ax = plt.subplots(figsize=(18, 5))
sns.lineplot(data=df_concatenated, x=df_concatenated.index, y='co2', ax=ax, linewidth=2)
ax.set_xlabel('Time Order', fontsize=12)
ax.set_ylabel('CO2 (PPM)', fontsize=12)
ax.set_title(f'CO2 Recordings Over Time ({TRATAMIENTO} PPM)', fontsize=14)
ax.grid(visible=True, linestyle='solid', alpha=0.5)
ax.margins(x=0.02, y=0.1)
plt.tight_layout()
plt.show()

In [ ]:
if(NORMAL == 'STANDARD'):
    scaler = StandardScaler()

    train_scaled = scaler.fit_transform(df_concatenated)
    test_scaled = scaler.transform(df_concatenated_nocap)
if(NORMAL == 'MINMAX'):
    scaler = MinMaxScaler()

    train_scaled = scaler.fit_transform(df_concatenated)
    test_scaled = scaler.transform(df_concatenated_nocap)
if(NORMAL == 'ROBUST'):
    scaler = RobustScaler()

    train_scaled = scaler.fit_transform(df_concatenated)
    test_scaled = scaler.transform(df_concatenated_nocap)


In [ ]:
TIME_STEPS = 3

def create_sequences(values, time_steps=TIME_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

train_sequenced = create_sequences(train_scaled)
print("Training input shape: ", train_sequenced.shape)

In [ ]:
model_1 = Sequential()
model_1.add(LSTM(16, activation='tanh', input_shape=(train_sequenced.shape[1], train_sequenced.shape[2]),return_sequences=False))
model_1.add(RepeatVector(train_sequenced.shape[1]))
model_1.add(LSTM(16, activation='tanh', return_sequences=True))
model_1.add(TimeDistributed(Dense(train_sequenced.shape[2])))

model_2 = Sequential()
model_2.add(LSTM(16, activation='tanh', input_shape=(train_sequenced.shape[1], train_sequenced.shape[2]), return_sequences=True))
model_2.add(Dropout(rate=0.2))
model_2.add(TimeDistributed(Dense(train_sequenced.shape[2])))

model_3 = Sequential()
model_3.add(LSTM(64, activation='tanh', input_shape=(train_sequenced.shape[1], train_sequenced.shape[2]), return_sequences=True))
model_3.add(Dropout(rate=0.2))
model_3.add(LSTM(16, activation='tanh', return_sequences=True))
model_3.add(TimeDistributed(Dense(train_sequenced.shape[2])))

model_4 = Sequential()
model_4.add(LSTM(128, activation='tanh', input_shape=(train_sequenced.shape[1], train_sequenced.shape[2]), return_sequences=True))
model_4.add(Dropout(rate=0.2))
model_4.add(LSTM(64, activation='tanh', return_sequences=True))
model_4.add(Dropout(rate=0.2))
model_4.add(LSTM(16, activation='tanh', return_sequences=True))
model_4.add(TimeDistributed(Dense(train_sequenced.shape[2])))

In [ ]:
LEARNING_RATE = 0.001
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=LOSS_FUNCTION)
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=LOSS_FUNCTION)
model_3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=LOSS_FUNCTION)
model_4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=LOSS_FUNCTION)

In [ ]:
EPOCHS = 400
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
history_1 = model_1.fit(train_sequenced, train_sequenced, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT, callbacks=[early_stopping_callback]).history

In [ ]:
history_2 = model_2.fit(train_sequenced, train_sequenced, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT, callbacks=[early_stopping_callback]).history

In [ ]:
history_3 = model_3.fit(train_sequenced, train_sequenced, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT, callbacks=[early_stopping_callback]).history

In [ ]:
history_4 = model_4.fit(train_sequenced, train_sequenced, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT, callbacks=[early_stopping_callback]).history

In [ ]:
sns.set_style('whitegrid')

histories = [history_1, history_2, history_3, history_4]
titles = ['Model 1', 'Model 2', 'Model 3', 'Model 4']

num_plots = len(histories)
num_plots_per_row = 1
num_rows = (num_plots + num_plots_per_row - 1) // num_plots_per_row

fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(12,12))
axes = axes.ravel()

for i, (history, title) in enumerate(zip(histories, titles)):
    ax = axes[i]
    ax.plot(history['loss'], label='Training loss')
    ax.plot(history['val_loss'], label='Validation loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title(title)
    ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
models = [model_1, model_2, model_3, model_4]

print("Train input shape: ", train_sequenced.shape)
train_reconstructed_sequences = [model.predict(train_sequenced) for model in models]
for i, sequence in enumerate(train_reconstructed_sequences):
    print(f'Recc Train input shape {i + 1}: ', sequence.shape)

test_sequenced = create_sequences(test_scaled)

print("Test input shape: ", test_sequenced.shape)
test_reconstructed_sequences = [model.predict(test_sequenced) for model in models]
for i, sequence in enumerate(test_reconstructed_sequences):
    print(f'Recc Test input shape {i + 1} : ', sequence.shape)

train_reconstructed_sequences1, train_reconstructed_sequences2, train_reconstructed_sequences3, train_reconstructed_sequences4 = train_reconstructed_sequences 
test_reconstructed_sequences1, test_reconstructed_sequences2, test_reconstructed_sequences3, test_reconstructed_sequences4 = test_reconstructed_sequences 


In [ ]:
anomalies_original = []
test_descaled = scaler.inverse_transform(test_scaled)
test_descaled_original = create_sequences(test_descaled)
anomalies = []

for sequence in test_descaled_original:
    if np.any(sequence > upper_limit):
        anomalies_original.append(True)
    else:
        anomalies_original.append(False)
anomalies_original = np.array(anomalies_original)

In [ ]:
sns.set_style('whitegrid')
fragment = 30
all_original = np.concatenate([train_sequenced[i] for i in range(fragment)])

num_plots_per_row = 2
num_rows = (len(train_reconstructed_sequences) + num_plots_per_row - 1) // num_plots_per_row

fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(12, 8))
axes = axes.ravel()

for i in range(len(train_reconstructed_sequences)):
    train_reconstructed_sequence = train_reconstructed_sequences[i]
    all_reconstructed = np.concatenate([train_reconstructed_sequence[j] for j in range(fragment)])
    ax = axes[i]
    ax.plot(all_original, label='Original')
    ax.plot(all_reconstructed, label='Reconstructed')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Value')
    ax.set_title(f'Comparison: Original vs Reconstructed {i + 1}')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
sns.set_style('whitegrid')
num_plots_per_row = 2
num_rows = (len(train_reconstructed_sequences) + num_plots_per_row - 1) // num_plots_per_row


fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(12, 8))
axes = axes.ravel()

thresholds = []
percentile = 99.5 #MODIFICABLE

for i in range(len(train_reconstructed_sequences)):
    train_reconstructed_sequence = train_reconstructed_sequences[i]
    if(LOSS1 == 'MAE'):
        train_loss = np.mean(np.abs(train_reconstructed_sequence - train_sequenced), axis=1) #MAE
        label = 'MAE'
    if(LOSS1 == 'MSE'):
        train_loss = np.mean(np.power(train_sequenced - train_reconstructed_sequence, 2), axis=1) #MSE
        label = 'MSE'
    threshold = np.percentile(train_loss, percentile)
    thresholds.append(threshold)
    
    ax = axes[i]
    ax.plot(train_loss, label=label + ' Loss')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Value')
    ax.set_title(f'{label} Loss Model {i + 1}')
    ax.legend()
    print(f'Reconstruction error threshold {i + 1} (Upper, {percentile}th percentile): ', threshold)

plt.tight_layout()
plt.show()

threshold1, threshold2, threshold3, threshold4 = thresholds

In [ ]:
anomalies_list = []

for i, (test_reconstructed_sequence, threshold) in enumerate(zip(test_reconstructed_sequences, thresholds)):
    if(LOSS2 == 'MAE'):
        test_loss = np.mean(np.abs(test_reconstructed_sequence - test_sequenced), axis=1) #MAE
    if(LOSS2 == 'MSE'):
        test_loss = np.mean(np.power(test_sequenced - test_reconstructed_sequence, 2), axis = 1 ) #MSE
    test_loss = test_loss.reshape((-1))

    anomalies = test_loss > threshold
    anomalies_list.append(anomalies)

anomalies1, anomalies2, anomalies3, anomalies4 = anomalies_list

In [ ]:
anomalies_list = [anomalies_original, anomalies1, anomalies2, anomalies3, anomalies4]

fig, axes = plt.subplots(len(anomalies_list), 1, figsize=(12, 5 * len(anomalies_list)))

for i, anomalies in enumerate(anomalies_list):
    ax = axes[i]
    ax.plot(test_descaled, label='Original values')
    anomaly_indices = np.where(anomalies)[0]
    ax.scatter(anomaly_indices, test_descaled[anomaly_indices], color='red', label='Anomalies')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('co2(PPM)')
    ax.set_title(f'Anomalies in Original Data model {i}')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
anomalies_list = [anomalies1, anomalies2, anomalies3, anomalies4]

def compute_metrics(original, predicted, model_num):
    original = np.array(original).astype(int)
    predicted = np.array(predicted).astype(int)
    
    accuracy = accuracy_score(original, predicted)
    f1 = f1_score(original, predicted)
    precision = precision_score(original, predicted)
    auc_roc = roc_auc_score(original, predicted)
    recall = recall_score(original, predicted)
    
    print(f'Model {model_num}', TRATAMIENTO, NORMAL, LOSS_FUNCTION, LOSS1, LOSS2, percentile, "Accuracy:", accuracy, "F1 Score:", f1, "Precision:", precision, "AUC-ROC:", auc_roc, "Recall:", recall)


for i, model_anomalies in enumerate(anomalies_list):
    compute_metrics(anomalies_original, model_anomalies, i+1)

In [ ]:
matrices_confusion = []

for model_anomalies in anomalies_list:
    confusion = confusion_matrix(anomalies_original, model_anomalies)
    matrices_confusion.append(confusion)

for i, confusion in enumerate(matrices_confusion):
    print(f"Modelo {i+1}:")
    print(confusion)
    print("\n")

labels = ['Normal', 'Anomalía']

num_rows = (len(matrices_confusion) + 1) // 2
num_cols = min(2, len(matrices_confusion))

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6 * num_rows))

for i, confusion in enumerate(matrices_confusion):
    row = i // num_cols
    col = i % num_cols
    df_confusion = pd.DataFrame(confusion, index=labels, columns=labels)
    ax = axes[row, col]
    sns.heatmap(df_confusion, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title(f"Matriz de confusión - Modelo {i+1}")
    ax.set_xlabel('Clase Predicha')
    ax.set_ylabel('Clase Real')
plt.subplots_adjust(wspace=0.4, hspace=0.4)
plt.show()



In [ ]:
models = [model_1, model_2, model_3, model_4]

for i, model in enumerate(models):
    model.save(f'model_{i+1}.h5')
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter._experimental_lower_tensor_list_ops = False
    tflite_model = converter.convert()

    with open(f'model_{i+1}.tflite', 'wb') as f:
        f.write(tflite_model)

In [ ]:
archivos = ['model_1.tflite', 'model_2.tflite', 'model_3.tflite', 'model_4.tflite']
for archivo in archivos:
    ruta_archivo = os.path.join('', archivo)
    tamaño_bytes = os.path.getsize(ruta_archivo)
    tamaño_kb = tamaño_bytes / 1024
    print(f"Tamaño de {archivo} en KB: {tamaño_kb} KB")

In [ ]:
model_lite = tf.lite.Interpreter(model_path='model_2.tflite')
model_lite.allocate_tensors()
input_details = model_lite.get_input_details()
output_details = model_lite.get_output_details()

input_data1 = test_sequenced[2709:2710].astype(np.float32)
model_lite.set_tensor(input_details[0]['index'], input_data1)
model_lite.invoke()
output_data = model_lite.get_tensor(output_details[0]['index'])
print(output_data)

test_loss = np.mean(np.power(input_data1 - output_data, 2), axis = 1 ) #MSE
test_loss = test_loss.reshape((-1))
anomaly = test_loss > threshold2
print("Es una anomalia? -> " , anomaly)

input_data = test_sequenced[2711:2712].astype(np.float32)
model_lite.set_tensor(input_details[0]['index'], input_data)
model_lite.invoke()
output_data = model_lite.get_tensor(output_details[0]['index'])
print(output_data)

test_loss = np.mean(np.power(input_data - output_data, 2), axis = 1 ) #MSE
test_loss = test_loss.reshape((-1))
anomaly1 = test_loss > threshold2
print("Es una anomalia? -> " , anomaly1)